In [1]:
import tensorflow as tf 
from tensorflow.keras.datasets.mnist import load_data
import numpy as np
import matplotlib.pyplot as plt
plt.figure(num=None, figsize=(30, 30), dpi=100, facecolor='w', edgecolor='k')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Flatten,Dropout,LeakyReLU,Reshape,Conv2DTranspose
from tensorflow.keras.optimizers import Adam

In [11]:
def discriminator_model(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
    
    return model

def generator_model(latent_dims=100):
    model = Sequential()
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, input_dim=latent_dims))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    # upsample to 14x14
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 28x28
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
    return model

def gan_model(g_model,d_model):
    
    d_model.trainable = False
    
    model=Sequential()
    model.add(g_model)
    model.add(d_model)
    
    opt = Adam(lr=0.0002,beta_1=0.5)
    
    model.compile(loss='binary_crossentropy',optimizer=opt)
    
    return model

def load_real_samples():
    
    (trainX,_),(_,_) = load_data()
    X = np.expand_dims(trainX,axis=-1)
    X = X.astype('float32')
    X = X/255.0
    return X

def generate_real_samples(dataset,batch_size):
    
    idxs = np.random.randint(0,dataset.shape[0],batch_size)
    X = dataset[idxs]
    y = np.ones((batch_size,1))
    return X,y

def generate_latent_points(latent_dims,batch_size):
    
    X = np.random.randn(batch_size*latent_dims)
    X = X.reshape((batch_size,latent_dims))
    return X

def generate_fake_samples(g_model,latent_dims,batch_size):
    
    latent_points = generate_latent_points(latent_dims,batch_size)
    X = g_model.predict(latent_points)
    y = np.zeros((batch_size,1))
    return X,y

def save_plot(examples, epoch, n=10):
    
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename = 'images/generation%03d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()

def summarize_performance(epoch,g_model,d_model,dataset,latent_dims,base_sample,batch_size=100):
    
    X_real,y_real = generate_real_samples(dataset,batch_size)
    X_fake,y_fake = generate_fake_samples(g_model,latent_dims,batch_size)
    
    _,real_accuracy = d_model.evaluate(X_real,y_real,verbose=0)
    _,fake_accuracy = d_model.evaluate(X_fake,y_fake,verbose=0)
    
    example = g_model.predict(base_sample)
    
    print("real_accuray:{} \t fake_accuray:{}".format(round(real_accuracy*100),round(fake_accuracy*100)))

    save_plot(example,epoch,n=int(np.sqrt(batch_size)))
    filename = 'models/generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)

def train(g_model,d_model,gan,dataset,latent_dims,base_sample,epochs=100,batch_size=256):
    
    batchs_per_epoch = dataset.shape[0]//batch_size
    half_batch_size = batch_size//2
    
    for epoch in range(epochs):
        
        for batch in range(batchs_per_epoch):
            X_real,y_real = generate_real_samples(dataset,half_batch_size)
            X_fake,y_fake = generate_fake_samples(g_model,latent_dims,half_batch_size)
            X , y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
            
            d_loss,_ = d_model.train_on_batch(X,y)
            
            X_gan = generate_latent_points(latent_dims,batch_size)
            y_gan = np.ones((batch_size,1))
            
            g_loss = gan.train_on_batch(X_gan,y_gan)
            
            if (batch+1)%1==0 :
                print("Epoch:{} \t Batch:{} \t d_loss: {} \t g_loss: {}".format(epoch,batch,round(d_loss,2),round(g_loss,2)))
                
        if (epoch+1)%1 == 0:
            
            summarize_performance(epoch,g_model,d_model,dataset,latent_dims,base_sample)


In [ ]:
latent_dims = 100
epochs = 2
batch_size = 256

g_model = generator_model()
d_model = discriminator_model()
gan = gan_model(g_model,d_model)

dataset = load_real_samples()
# dataset = dataset[:1000]

X,y = generate_fake_samples(g_model,latent_dims,128)
base_sample = np.load("base_sample.npy")


train(g_model,d_model,gan,dataset,latent_dims,base_sample,epochs,batch_size)